In [1]:
import numpy as np
import pandas as pd
import os
import requests
import bs4
import json
import seaborn as sns
import re
from sklearn.compose import ColumnTransformer

In [2]:
url = 'https://lol.fandom.com/wiki/Leaguepedia:News/{0}/{1}'

In [3]:
jan_2021 = url.format(2021, 'January')
response = requests.get(jan_2021)

In [4]:
soup = bs4.BeautifulSoup(response.content)

In [5]:
rows = soup.find_all('tr')

In [6]:
rows[22].text.strip('edit')

'January 31'

In [7]:
region = rows[23].find('td').text
team_name = rows[23].find_all('td')[1].find('span', attrs={'class': 'teamname'}).text
news = rows[23].find_all('td')[2].text

In [8]:
news

'Czaru, Bolszak, Dawidsonek, Ejsner, and Grk join.'

In [9]:
re.split('[ ]*join[s\., ]*', news)

['Czaru, Bolszak, Dawidsonek, Ejsner, and Grk', '']

In [10]:
path = os.path.join('data', '2020_matches.csv')
matches = pd.read_csv(path)
matches['team'] = matches['team'].replace('DAMWON Gaming', 'DWG KIA')

/Users/flynnosullivan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
matches['league'].value_counts()
lol = matches[matches['league'].isin (['LPL', 'LCK', 'LCS', 'LEC'])]

In [12]:
only_more_than_10 = (lol['player'].value_counts() >= 5)
exp_players = only_more_than_10[only_more_than_10].index.to_list()
only_exp = lol[lol['player'].isin(exp_players)]

In [13]:
def helper_team(row):
    name = row['player']
    teams = matches[matches['player'] == name]['team'].value_counts().index.to_list()
    return teams

def helper_pos(row):
    name = row['player']
    position = matches[matches['player'] == name]['position'].value_counts().idxmax()
    return position

def helper_games_played(row):
    name = row['player']
    num_matches = len(lol[lol['player'] == name])
    return num_matches

df = pd.DataFrame(data={'player': only_exp['player'].value_counts().index}, index=only_exp['player'].value_counts().index)
df['team'] = df.apply(helper_team, axis=1)
df['position'] = df.apply(helper_pos, axis=1)
df['games_played'] = df.apply(helper_games_played, axis=1)

In [14]:
df['win_rate'] = only_exp.groupby('player')['result'].mean()
df['avg_kills'] = only_exp.groupby('player')['kills'].mean()
df['avg_deaths'] = only_exp.groupby('player')['deaths'].mean()
df['avg_assists'] = only_exp.groupby('player')['assists'].mean()
df['avg_teamkills'] = only_exp.groupby('player')['teamkills'].mean()
df['avg_teamdeaths'] = only_exp.groupby('player')['teamdeaths'].mean()
df['avg_doublekills'] = only_exp.groupby('player')['doublekills'].mean()
df['avg_triplekills'] = only_exp.groupby('player')['triplekills'].mean()
df['avg_quadrakills'] = only_exp.groupby('player')['quadrakills'].mean()
df['avg_pentakills'] = only_exp.groupby('player')['pentakills'].mean()
df['firstblood_rate'] = only_exp.groupby('player')['firstblood'].mean()
df['firstbloodvictim_rate'] = only_exp.groupby('player')['firstbloodvictim'].mean()
df['avg_team_kpm'] = only_exp.groupby('player')['team kpm'].mean()
df['avg_ckpm'] = only_exp.groupby('player')['ckpm'].mean()
df['avg_dmg_to_champions'] = only_exp.groupby('player')['damagetochampions'].mean()
df['avg_dpm'] = only_exp.groupby('player')['dpm'].mean()
df['avg_dmgshare'] = only_exp.groupby('player')['damageshare'].mean()
df['avg_dmgmitigatedperminute'] = only_exp.groupby('player')['damagemitigatedperminute'].mean()
df['avg_wardsplaced'] = only_exp.groupby('player')['wardsplaced'].mean()
df['avg_wpm'] = only_exp.groupby('player')['wpm'].mean()
df['avg_wardskilled'] = only_exp.groupby('player')['wardskilled'].mean()
df['avg_controlwardsbought'] = only_exp.groupby('player')['controlwardsbought'].mean()
df['avg_visionscore'] = only_exp.groupby('player')['visionscore'].mean()
df['avg_totalgold'] = only_exp.groupby('player')['totalgold'].mean()
df['avg_earnedgold'] = only_exp.groupby('player')['earnedgold'].mean()
df['avg_earnedgpm'] = only_exp.groupby('player')['earned gpm'].mean()
df['avg_earnedgoldshare'] = only_exp.groupby('player')['earnedgoldshare'].mean()
df['avg_goldspent'] = only_exp.groupby('player')['goldspent'].mean()
df['avg_cs'] = only_exp.groupby('player')['total cs'].mean()
df['avg_minionkills'] = only_exp.groupby('player')['minionkills'].mean()
df['avg_monsterkillsownjg'] = only_exp.groupby('player')['monsterkillsownjungle'].mean()
df['avg_monsterkillsenemyjg'] = only_exp.groupby('player')['monsterkillsenemyjungle'].mean()
df['avg_cspm'] = only_exp.groupby('player')['cspm'].mean()
df['avg_goldat10'] = only_exp.groupby('player')['goldat10'].mean()
df['avg_xpat10'] = only_exp.groupby('player')['xpat10'].mean()
df['avg_csat10'] = only_exp.groupby('player')['csat10'].mean()
df['avg_golddiffat10'] = only_exp.groupby('player')['golddiffat10'].mean()
df['avg_xpdiffat10'] = only_exp.groupby('player')['xpdiffat10'].mean()
df['avg_csdiffat10'] = only_exp.groupby('player')['csdiffat10'].mean()
df['avg_killsat10'] = only_exp.groupby('player')['killsat10'].mean()
df['avg_assistsat10'] = only_exp.groupby('player')['assistsat10'].mean()
df['avg_deathsat10'] = only_exp.groupby('player')['deathsat10'].mean()
df['avg_goldat15'] = only_exp.groupby('player')['goldat15'].mean()
df['avg_xpat15'] = only_exp.groupby('player')['xpat15'].mean()
df['avg_csat15'] = only_exp.groupby('player')['csat15'].mean()
df['avg_golddiffat15'] = only_exp.groupby('player')['golddiffat15'].mean()
df['avg_xpdiffat15'] = only_exp.groupby('player')['xpdiffat15'].mean()
df['avg_csdiffat15'] = only_exp.groupby('player')['csdiffat15'].mean()
df['avg_killsat15'] = only_exp.groupby('player')['killsat15'].mean()
df['avg_assistsat15'] = only_exp.groupby('player')['assistsat15'].mean()
df['avg_deathsat15'] = only_exp.groupby('player')['deathsat15'].mean()

In [15]:
mids_1 = df.loc[df['position'] == 'mid', :]
jgs_1 = df.loc[df['position'] == 'jng', :]
tops_1 = df.loc[df['position'] == 'top', :]
adcs_1 = df.loc[df['position'] == 'bot', :]
sups_1 = df.loc[df['position'] == 'sup', :]

In [16]:
mids = mids_1.copy()
jgs = jgs_1.copy()
tops = tops_1.copy()
adcs = adcs_1.copy()
sups = sups_1.copy()

In [17]:
top_win_rate_std = (tops['win_rate'] - np.mean(tops['win_rate'])) / np.std(tops['win_rate'])
top_kills_std = (tops['avg_kills'] - np.mean(tops['avg_kills'])) / np.std(tops['avg_kills'])
top_deaths_std = (tops['avg_deaths'] - np.mean(tops['avg_deaths'])) / np.std(tops['avg_deaths'])
top_assists_std = (tops['avg_assists'] - np.mean(tops['avg_assists'])) / np.std(tops['avg_assists'])
top_first_blood_rate_std = (tops['firstblood_rate'] - np.mean(tops['firstblood_rate'])) / np.std(tops['firstblood_rate'])
top_first_blood_victim_rate_std = (tops['firstbloodvictim_rate'] - np.mean(tops['firstbloodvictim_rate'])) / np.std(tops['firstbloodvictim_rate'])
top_kpm_std = (tops['avg_ckpm'] - np.mean(tops['avg_ckpm'])) / np.std(tops['avg_ckpm'])
top_dmg_std = (tops['avg_dmg_to_champions'] - np.mean(tops['avg_dmg_to_champions'])) / np.std(tops['avg_dmg_to_champions'])
top_dpm_std = (tops['avg_dpm'] - np.mean(tops['avg_dpm'])) / np.std(tops['avg_dpm'])
top_dmg_mitigated_std = (tops['avg_dmgmitigatedperminute'] - np.mean(tops['avg_dmgmitigatedperminute'])) / np.std(tops['avg_dmgmitigatedperminute'])
top_wards_placed_std = (tops['avg_wardsplaced'] - np.mean(tops['avg_wardsplaced'])) / np.std(tops['avg_wardsplaced'])
top_wpm_std = (tops['avg_wpm'] - np.mean(tops['avg_wpm'])) / np.std(tops['avg_wpm'])
top_wards_killed_std = (tops['avg_wardskilled'] - np.mean(tops['avg_wardskilled'])) / np.std(tops['avg_wardskilled'])
top_ctrl_wards_std = (tops['avg_controlwardsbought'] - np.mean(tops['avg_controlwardsbought'])) / np.std(tops['avg_controlwardsbought'])
top_vision_score_std = (tops['avg_visionscore'] - np.mean(tops['avg_visionscore'])) / np.std(tops['avg_visionscore'])
top_total_gold_std = (tops['avg_totalgold'] - np.mean(tops['avg_totalgold'])) / np.std(tops['avg_totalgold'])
top_earned_gold_std = (tops['avg_earnedgold'] - np.mean(tops['avg_earnedgold'])) / np.std(tops['avg_earnedgold'])
top_earned_gpm_std = (tops['avg_earnedgpm'] - np.mean(tops['avg_earnedgpm'])) / np.std(tops['avg_earnedgpm'])
top_gold_share_std = (tops['avg_earnedgoldshare'] - np.mean(tops['avg_earnedgoldshare'])) / np.std(tops['avg_earnedgoldshare'])
top_gold_spent_std = (tops['avg_goldspent'] - np.mean(tops['avg_goldspent'])) / np.std(tops['avg_goldspent'])
top_cs_std = (tops['avg_cs'] - np.mean(tops['avg_cs'])) / np.std(tops['avg_cs'])
top_minion_kills_std = (tops['avg_minionkills'] - np.mean(tops['avg_minionkills'])) / np.std(tops['avg_minionkills'])
top_cspm_std = (tops['avg_cspm'] - np.mean(tops['avg_cspm'])) / np.std(tops['avg_cspm'])
top_goldat10_std = (tops['avg_goldat10'] - np.mean(tops['avg_goldat10'])) / np.std(tops['avg_goldat10'])
top_xpat10_std = (tops['avg_xpat10'] - np.mean(tops['avg_xpat10'])) / np.std(tops['avg_xpat10'])
top_csat10_std = (tops['avg_csat10'] - np.mean(tops['avg_csat10'])) / np.std(tops['avg_csat10'])
top_golddiffat10_std = (tops['avg_golddiffat10'] - np.mean(tops['avg_golddiffat10'])) / np.std(tops['avg_golddiffat10'])
top_xpdiffat10_std = (tops['avg_xpdiffat10'] - np.mean(tops['avg_xpdiffat10'])) / np.std(tops['avg_xpdiffat10'])
top_csdiffat10_std = (tops['avg_csdiffat10'] - np.mean(tops['avg_csdiffat10'])) / np.std(tops['avg_csdiffat10'])
top_killsat10_std = (tops['avg_killsat10'] - np.mean(tops['avg_killsat10'])) / np.std(tops['avg_killsat10'])
top_assistsat10_std = (tops['avg_assistsat10'] - np.mean(tops['avg_assistsat10'])) / np.std(tops['avg_assistsat10'])
top_deathsat10_std = (tops['avg_deathsat10'] - np.mean(tops['avg_deathsat10'])) / np.std(tops['avg_deathsat10'])
top_goldat15_std = (tops['avg_goldat15'] - np.mean(tops['avg_goldat15'])) / np.std(tops['avg_goldat15'])
top_xpat15_std = (tops['avg_xpat15'] - np.mean(tops['avg_xpat15'])) / np.std(tops['avg_xpat15'])
top_csat15_std = (tops['avg_csat15'] - np.mean(tops['avg_csat15'])) / np.std(tops['avg_csat15'])
top_golddiffat15_std = (tops['avg_golddiffat15'] - np.mean(tops['avg_golddiffat15'])) / np.std(tops['avg_golddiffat15'])
top_xpdiffat15_std = (tops['avg_xpdiffat15'] - np.mean(tops['avg_xpdiffat15'])) / np.std(tops['avg_xpdiffat15'])
top_csdiffat15_std = (tops['avg_csdiffat15'] - np.mean(tops['avg_csdiffat15'])) / np.std(tops['avg_csdiffat15'])
top_killsat15_std = (tops['avg_killsat15'] - np.mean(tops['avg_killsat15'])) / np.std(tops['avg_killsat15'])
top_assistsat15_std = (tops['avg_assistsat15'] - np.mean(tops['avg_assistsat15'])) / np.std(tops['avg_assistsat15'])
top_deathsat15_std = (tops['avg_deathsat15'] - np.mean(tops['avg_deathsat15'])) / np.std(tops['avg_deathsat15'])

In [18]:
mid_win_rate_std = (mids['win_rate'] - np.mean(mids['win_rate'])) / np.std(mids['win_rate'])
mid_kills_std = (mids['avg_kills'] - np.mean(mids['avg_kills'])) / np.std(mids['avg_kills'])
mid_deaths_std = (mids['avg_deaths'] - np.mean(mids['avg_deaths'])) / np.std(mids['avg_deaths'])
mid_assists_std = (mids['avg_assists'] - np.mean(mids['avg_assists'])) / np.std(mids['avg_assists'])
mid_first_blood_rate_std = (mids['firstblood_rate'] - np.mean(mids['firstblood_rate'])) / np.std(mids['firstblood_rate'])
mid_first_blood_victim_rate_std = (mids['firstbloodvictim_rate'] - np.mean(mids['firstbloodvictim_rate'])) / np.std(mids['firstbloodvictim_rate'])
mid_kpm_std = (mids['avg_ckpm'] - np.mean(mids['avg_ckpm'])) / np.std(mids['avg_ckpm'])
mid_dmg_std = (mids['avg_dmg_to_champions'] - np.mean(mids['avg_dmg_to_champions'])) / np.std(mids['avg_dmg_to_champions'])
mid_dpm_std = (mids['avg_dpm'] - np.mean(mids['avg_dpm'])) / np.std(mids['avg_dpm'])
mid_dmg_mitigated_std = (mids['avg_dmgmitigatedperminute'] - np.mean(mids['avg_dmgmitigatedperminute'])) / np.std(mids['avg_dmgmitigatedperminute'])
mid_wards_placed_std = (mids['avg_wardsplaced'] - np.mean(mids['avg_wardsplaced'])) / np.std(mids['avg_wardsplaced'])
mid_wpm_std = (mids['avg_wpm'] - np.mean(mids['avg_wpm'])) / np.std(mids['avg_wpm'])
mid_wards_killed_std = (mids['avg_wardskilled'] - np.mean(mids['avg_wardskilled'])) / np.std(mids['avg_wardskilled'])
mid_ctrl_wards_std = (mids['avg_controlwardsbought'] - np.mean(mids['avg_controlwardsbought'])) / np.std(mids['avg_controlwardsbought'])
mid_vision_score_std = (mids['avg_visionscore'] - np.mean(mids['avg_visionscore'])) / np.std(mids['avg_visionscore'])
mid_total_gold_std = (mids['avg_totalgold'] - np.mean(mids['avg_totalgold'])) / np.std(mids['avg_totalgold'])
mid_earned_gold_std = (mids['avg_earnedgold'] - np.mean(mids['avg_earnedgold'])) / np.std(mids['avg_earnedgold'])
mid_earned_gpm_std = (mids['avg_earnedgpm'] - np.mean(mids['avg_earnedgpm'])) / np.std(mids['avg_earnedgpm'])
mid_gold_share_std = (mids['avg_earnedgoldshare'] - np.mean(mids['avg_earnedgoldshare'])) / np.std(mids['avg_earnedgoldshare'])
mid_gold_spent_std = (mids['avg_goldspent'] - np.mean(mids['avg_goldspent'])) / np.std(mids['avg_goldspent'])
mid_cs_std = (mids['avg_cs'] - np.mean(mids['avg_cs'])) / np.std(mids['avg_cs'])
mid_minion_kills_std = (mids['avg_minionkills'] - np.mean(mids['avg_minionkills'])) / np.std(mids['avg_minionkills'])
mid_cspm_std = (mids['avg_cspm'] - np.mean(mids['avg_cspm'])) / np.std(mids['avg_cspm'])
mid_goldat10_std = (mids['avg_goldat10'] - np.mean(mids['avg_goldat10'])) / np.std(mids['avg_goldat10'])
mid_xpat10_std = (mids['avg_xpat10'] - np.mean(mids['avg_xpat10'])) / np.std(mids['avg_xpat10'])
mid_csat10_std = (mids['avg_csat10'] - np.mean(mids['avg_csat10'])) / np.std(mids['avg_csat10'])
mid_golddiffat10_std = (mids['avg_golddiffat10'] - np.mean(mids['avg_golddiffat10'])) / np.std(mids['avg_golddiffat10'])
mid_xpdiffat10_std = (mids['avg_xpdiffat10'] - np.mean(mids['avg_xpdiffat10'])) / np.std(mids['avg_xpdiffat10'])
mid_csdiffat10_std = (mids['avg_csdiffat10'] - np.mean(mids['avg_csdiffat10'])) / np.std(mids['avg_csdiffat10'])
mid_killsat10_std = (mids['avg_killsat10'] - np.mean(mids['avg_killsat10'])) / np.std(mids['avg_killsat10'])
mid_assistsat10_std = (mids['avg_assistsat10'] - np.mean(mids['avg_assistsat10'])) / np.std(mids['avg_assistsat10'])
mid_deathsat10_std = (mids['avg_deathsat10'] - np.mean(mids['avg_deathsat10'])) / np.std(mids['avg_deathsat10'])
mid_goldat15_std = (mids['avg_goldat15'] - np.mean(mids['avg_goldat15'])) / np.std(mids['avg_goldat15'])
mid_xpat15_std = (mids['avg_xpat15'] - np.mean(mids['avg_xpat15'])) / np.std(mids['avg_xpat15'])
mid_csat15_std = (mids['avg_csat15'] - np.mean(mids['avg_csat15'])) / np.std(mids['avg_csat15'])
mid_golddiffat15_std = (mids['avg_golddiffat15'] - np.mean(mids['avg_golddiffat15'])) / np.std(mids['avg_golddiffat15'])
mid_xpdiffat15_std = (mids['avg_xpdiffat15'] - np.mean(mids['avg_xpdiffat15'])) / np.std(mids['avg_xpdiffat15'])
mid_csdiffat15_std = (mids['avg_csdiffat15'] - np.mean(mids['avg_csdiffat15'])) / np.std(mids['avg_csdiffat15'])
mid_killsat15_std = (mids['avg_killsat15'] - np.mean(mids['avg_killsat15'])) / np.std(mids['avg_killsat15'])
mid_assistsat15_std = (mids['avg_assistsat15'] - np.mean(mids['avg_assistsat15'])) / np.std(mids['avg_assistsat15'])
mid_deathsat15_std = (mids['avg_deathsat15'] - np.mean(mids['avg_deathsat15'])) / np.std(mids['avg_deathsat15'])

In [19]:
jg_win_rate_std = (jgs['win_rate'] - np.mean(jgs['win_rate'])) / np.std(jgs['win_rate'])
jg_kills_std = (jgs['avg_kills'] - np.mean(jgs['avg_kills'])) / np.std(jgs['avg_kills'])
jg_deaths_std = (jgs['avg_deaths'] - np.mean(jgs['avg_deaths'])) / np.std(jgs['avg_deaths'])
jg_assists_std = (jgs['avg_assists'] - np.mean(jgs['avg_assists'])) / np.std(jgs['avg_assists'])
jg_first_blood_rate_std = (jgs['firstblood_rate'] - np.mean(jgs['firstblood_rate'])) / np.std(jgs['firstblood_rate'])
jg_first_blood_victim_rate_std = (jgs['firstbloodvictim_rate'] - np.mean(jgs['firstbloodvictim_rate'])) / np.std(jgs['firstbloodvictim_rate'])
jg_kpm_std = (jgs['avg_ckpm'] - np.mean(jgs['avg_ckpm'])) / np.std(jgs['avg_ckpm'])
jg_dmg_std = (jgs['avg_dmg_to_champions'] - np.mean(jgs['avg_dmg_to_champions'])) / np.std(jgs['avg_dmg_to_champions'])
jg_dpm_std = (jgs['avg_dpm'] - np.mean(jgs['avg_dpm'])) / np.std(jgs['avg_dpm'])
jg_dmg_mitigated_std = (jgs['avg_dmgmitigatedperminute'] - np.mean(jgs['avg_dmgmitigatedperminute'])) / np.std(jgs['avg_dmgmitigatedperminute'])
jg_wards_placed_std = (jgs['avg_wardsplaced'] - np.mean(jgs['avg_wardsplaced'])) / np.std(jgs['avg_wardsplaced'])
jg_wpm_std = (jgs['avg_wpm'] - np.mean(jgs['avg_wpm'])) / np.std(jgs['avg_wpm'])
jg_wards_killed_std = (jgs['avg_wardskilled'] - np.mean(jgs['avg_wardskilled'])) / np.std(jgs['avg_wardskilled'])
jg_ctrl_wards_std = (jgs['avg_controlwardsbought'] - np.mean(jgs['avg_controlwardsbought'])) / np.std(jgs['avg_controlwardsbought'])
jg_vision_score_std = (jgs['avg_visionscore'] - np.mean(jgs['avg_visionscore'])) / np.std(jgs['avg_visionscore'])
jg_total_gold_std = (jgs['avg_totalgold'] - np.mean(jgs['avg_totalgold'])) / np.std(jgs['avg_totalgold'])
jg_earned_gold_std = (jgs['avg_earnedgold'] - np.mean(jgs['avg_earnedgold'])) / np.std(jgs['avg_earnedgold'])
jg_earned_gpm_std = (jgs['avg_earnedgpm'] - np.mean(jgs['avg_earnedgpm'])) / np.std(jgs['avg_earnedgpm'])
jg_gold_share_std = (jgs['avg_earnedgoldshare'] - np.mean(jgs['avg_earnedgoldshare'])) / np.std(jgs['avg_earnedgoldshare'])
jg_gold_spent_std = (jgs['avg_goldspent'] - np.mean(jgs['avg_goldspent'])) / np.std(jgs['avg_goldspent'])
jg_cs_std = (jgs['avg_cs'] - np.mean(jgs['avg_cs'])) / np.std(jgs['avg_cs'])
jg_minion_kills_std = (jgs['avg_minionkills'] - np.mean(jgs['avg_minionkills'])) / np.std(jgs['avg_minionkills'])
jg_cspm_std = (jgs['avg_cspm'] - np.mean(jgs['avg_cspm'])) / np.std(jgs['avg_cspm'])
jg_goldat10_std = (jgs['avg_goldat10'] - np.mean(jgs['avg_goldat10'])) / np.std(jgs['avg_goldat10'])
jg_xpat10_std = (jgs['avg_xpat10'] - np.mean(jgs['avg_xpat10'])) / np.std(jgs['avg_xpat10'])
jg_csat10_std = (jgs['avg_csat10'] - np.mean(jgs['avg_csat10'])) / np.std(jgs['avg_csat10'])
jg_golddiffat10_std = (jgs['avg_golddiffat10'] - np.mean(jgs['avg_golddiffat10'])) / np.std(jgs['avg_golddiffat10'])
jg_xpdiffat10_std = (jgs['avg_xpdiffat10'] - np.mean(jgs['avg_xpdiffat10'])) / np.std(jgs['avg_xpdiffat10'])
jg_csdiffat10_std = (jgs['avg_csdiffat10'] - np.mean(jgs['avg_csdiffat10'])) / np.std(jgs['avg_csdiffat10'])
jg_killsat10_std = (jgs['avg_killsat10'] - np.mean(jgs['avg_killsat10'])) / np.std(jgs['avg_killsat10'])
jg_assistsat10_std = (jgs['avg_assistsat10'] - np.mean(jgs['avg_assistsat10'])) / np.std(jgs['avg_assistsat10'])
jg_deathsat10_std = (jgs['avg_deathsat10'] - np.mean(jgs['avg_deathsat10'])) / np.std(jgs['avg_deathsat10'])
jg_goldat15_std = (jgs['avg_goldat15'] - np.mean(jgs['avg_goldat15'])) / np.std(jgs['avg_goldat15'])
jg_xpat15_std = (jgs['avg_xpat15'] - np.mean(jgs['avg_xpat15'])) / np.std(jgs['avg_xpat15'])
jg_csat15_std = (jgs['avg_csat15'] - np.mean(jgs['avg_csat15'])) / np.std(jgs['avg_csat15'])
jg_golddiffat15_std = (jgs['avg_golddiffat15'] - np.mean(jgs['avg_golddiffat15'])) / np.std(jgs['avg_golddiffat15'])
jg_xpdiffat15_std = (jgs['avg_xpdiffat15'] - np.mean(jgs['avg_xpdiffat15'])) / np.std(jgs['avg_xpdiffat15'])
jg_csdiffat15_std = (jgs['avg_csdiffat15'] - np.mean(jgs['avg_csdiffat15'])) / np.std(jgs['avg_csdiffat15'])
jg_killsat15_std = (jgs['avg_killsat15'] - np.mean(jgs['avg_killsat15'])) / np.std(jgs['avg_killsat15'])
jg_assistsat15_std = (jgs['avg_assistsat15'] - np.mean(jgs['avg_assistsat15'])) / np.std(jgs['avg_assistsat15'])
jg_deathsat15_std = (jgs['avg_deathsat15'] - np.mean(jgs['avg_deathsat15'])) / np.std(jgs['avg_deathsat15'])
jg_monsterkillsownjung_std = (jgs['avg_monsterkillsownjg'] - np.mean(jgs['avg_monsterkillsownjg'])) / np.std(jgs['avg_monsterkillsownjg'])
jg_monsterkillsenemyjung_std = (jgs['avg_monsterkillsenemyjg'] - np.mean(jgs['avg_monsterkillsenemyjg'])) / np.std(jgs['avg_monsterkillsenemyjg'])

In [20]:
sup_win_rate_std = (sups['win_rate'] - np.mean(sups['win_rate'])) / np.std(sups['win_rate'])
sup_kills_std = (sups['avg_kills'] - np.mean(sups['avg_kills'])) / np.std(sups['avg_kills'])
sup_deaths_std = (sups['avg_deaths'] - np.mean(sups['avg_deaths'])) / np.std(sups['avg_deaths'])
sup_assists_std = (sups['avg_assists'] - np.mean(sups['avg_assists'])) / np.std(sups['avg_assists'])
sup_first_blood_rate_std = (sups['firstblood_rate'] - np.mean(sups['firstblood_rate'])) / np.std(sups['firstblood_rate'])
sup_first_blood_victim_rate_std = (sups['firstbloodvictim_rate'] - np.mean(sups['firstbloodvictim_rate'])) / np.std(sups['firstbloodvictim_rate'])
sup_kpm_std = (sups['avg_ckpm'] - np.mean(sups['avg_ckpm'])) / np.std(sups['avg_ckpm'])
sup_dmg_std = (sups['avg_dmg_to_champions'] - np.mean(sups['avg_dmg_to_champions'])) / np.std(sups['avg_dmg_to_champions'])
sup_dpm_std = (sups['avg_dpm'] - np.mean(sups['avg_dpm'])) / np.std(sups['avg_dpm'])
sup_dmg_mitigated_std = (sups['avg_dmgmitigatedperminute'] - np.mean(sups['avg_dmgmitigatedperminute'])) / np.std(sups['avg_dmgmitigatedperminute'])
sup_wards_placed_std = (sups['avg_wardsplaced'] - np.mean(sups['avg_wardsplaced'])) / np.std(sups['avg_wardsplaced'])
sup_wpm_std = (sups['avg_wpm'] - np.mean(sups['avg_wpm'])) / np.std(sups['avg_wpm'])
sup_wards_killed_std = (sups['avg_wardskilled'] - np.mean(sups['avg_wardskilled'])) / np.std(sups['avg_wardskilled'])
sup_ctrl_wards_std = (sups['avg_controlwardsbought'] - np.mean(sups['avg_controlwardsbought'])) / np.std(sups['avg_controlwardsbought'])
sup_vision_score_std = (sups['avg_visionscore'] - np.mean(sups['avg_visionscore'])) / np.std(sups['avg_visionscore'])
sup_total_gold_std = (sups['avg_totalgold'] - np.mean(sups['avg_totalgold'])) / np.std(sups['avg_totalgold'])
sup_earned_gold_std = (sups['avg_earnedgold'] - np.mean(sups['avg_earnedgold'])) / np.std(sups['avg_earnedgold'])
sup_earned_gpm_std = (sups['avg_earnedgpm'] - np.mean(sups['avg_earnedgpm'])) / np.std(sups['avg_earnedgpm'])
sup_gold_share_std = (sups['avg_earnedgoldshare'] - np.mean(sups['avg_earnedgoldshare'])) / np.std(sups['avg_earnedgoldshare'])
sup_gold_spent_std = (sups['avg_goldspent'] - np.mean(sups['avg_goldspent'])) / np.std(sups['avg_goldspent'])
sup_cs_std = (sups['avg_cs'] - np.mean(sups['avg_cs'])) / np.std(sups['avg_cs'])
sup_minion_kills_std = (sups['avg_minionkills'] - np.mean(sups['avg_minionkills'])) / np.std(sups['avg_minionkills'])
sup_cspm_std = (sups['avg_cspm'] - np.mean(sups['avg_cspm'])) / np.std(sups['avg_cspm'])
sup_goldat10_std = (sups['avg_goldat10'] - np.mean(sups['avg_goldat10'])) / np.std(sups['avg_goldat10'])
sup_xpat10_std = (sups['avg_xpat10'] - np.mean(sups['avg_xpat10'])) / np.std(sups['avg_xpat10'])
sup_csat10_std = (sups['avg_csat10'] - np.mean(sups['avg_csat10'])) / np.std(sups['avg_csat10'])
sup_golddiffat10_std = (sups['avg_golddiffat10'] - np.mean(sups['avg_golddiffat10'])) / np.std(sups['avg_golddiffat10'])
sup_xpdiffat10_std = (sups['avg_xpdiffat10'] - np.mean(sups['avg_xpdiffat10'])) / np.std(sups['avg_xpdiffat10'])
sup_csdiffat10_std = (sups['avg_csdiffat10'] - np.mean(sups['avg_csdiffat10'])) / np.std(sups['avg_csdiffat10'])
sup_killsat10_std = (sups['avg_killsat10'] - np.mean(sups['avg_killsat10'])) / np.std(sups['avg_killsat10'])
sup_assistsat10_std = (sups['avg_assistsat10'] - np.mean(sups['avg_assistsat10'])) / np.std(sups['avg_assistsat10'])
sup_deathsat10_std = (sups['avg_deathsat10'] - np.mean(sups['avg_deathsat10'])) / np.std(sups['avg_deathsat10'])
sup_goldat15_std = (sups['avg_goldat15'] - np.mean(sups['avg_goldat15'])) / np.std(sups['avg_goldat15'])
sup_xpat15_std = (sups['avg_xpat15'] - np.mean(sups['avg_xpat15'])) / np.std(sups['avg_xpat15'])
sup_csat15_std = (sups['avg_csat15'] - np.mean(sups['avg_csat15'])) / np.std(sups['avg_csat15'])
sup_golddiffat15_std = (sups['avg_golddiffat15'] - np.mean(sups['avg_golddiffat15'])) / np.std(sups['avg_golddiffat15'])
sup_xpdiffat15_std = (sups['avg_xpdiffat15'] - np.mean(sups['avg_xpdiffat15'])) / np.std(sups['avg_xpdiffat15'])
sup_csdiffat15_std = (sups['avg_csdiffat15'] - np.mean(sups['avg_csdiffat15'])) / np.std(sups['avg_csdiffat15'])
sup_killsat15_std = (sups['avg_killsat15'] - np.mean(sups['avg_killsat15'])) / np.std(sups['avg_killsat15'])
sup_assistsat15_std = (sups['avg_assistsat15'] - np.mean(sups['avg_assistsat15'])) / np.std(sups['avg_assistsat15'])
sup_deathsat15_std = (sups['avg_deathsat15'] - np.mean(sups['avg_deathsat15'])) / np.std(sups['avg_deathsat15'])

In [21]:
adc_win_rate_std = (adcs['win_rate'] - np.mean(adcs['win_rate'])) / np.std(adcs['win_rate'])
adc_kills_std = (adcs['avg_kills'] - np.mean(adcs['avg_kills'])) / np.std(adcs['avg_kills'])
adc_deaths_std = (adcs['avg_deaths'] - np.mean(adcs['avg_deaths'])) / np.std(adcs['avg_deaths'])
adc_assists_std = (adcs['avg_assists'] - np.mean(adcs['avg_assists'])) / np.std(adcs['avg_assists'])
adc_first_blood_rate_std = (adcs['firstblood_rate'] - np.mean(adcs['firstblood_rate'])) / np.std(adcs['firstblood_rate'])
adc_first_blood_victim_rate_std = (adcs['firstbloodvictim_rate'] - np.mean(adcs['firstbloodvictim_rate'])) / np.std(adcs['firstbloodvictim_rate'])
adc_kpm_std = (adcs['avg_ckpm'] - np.mean(adcs['avg_ckpm'])) / np.std(adcs['avg_ckpm'])
adc_dmg_std = (adcs['avg_dmg_to_champions'] - np.mean(adcs['avg_dmg_to_champions'])) / np.std(adcs['avg_dmg_to_champions'])
adc_dpm_std = (adcs['avg_dpm'] - np.mean(adcs['avg_dpm'])) / np.std(adcs['avg_dpm'])
adc_dmg_mitigated_std = (adcs['avg_dmgmitigatedperminute'] - np.mean(adcs['avg_dmgmitigatedperminute'])) / np.std(adcs['avg_dmgmitigatedperminute'])
adc_wards_placed_std = (adcs['avg_wardsplaced'] - np.mean(adcs['avg_wardsplaced'])) / np.std(adcs['avg_wardsplaced'])
adc_wpm_std = (adcs['avg_wpm'] - np.mean(adcs['avg_wpm'])) / np.std(adcs['avg_wpm'])
adc_wards_killed_std = (adcs['avg_wardskilled'] - np.mean(adcs['avg_wardskilled'])) / np.std(adcs['avg_wardskilled'])
adc_ctrl_wards_std = (adcs['avg_controlwardsbought'] - np.mean(adcs['avg_controlwardsbought'])) / np.std(adcs['avg_controlwardsbought'])
adc_vision_score_std = (adcs['avg_visionscore'] - np.mean(adcs['avg_visionscore'])) / np.std(adcs['avg_visionscore'])
adc_total_gold_std = (adcs['avg_totalgold'] - np.mean(adcs['avg_totalgold'])) / np.std(adcs['avg_totalgold'])
adc_earned_gold_std = (adcs['avg_earnedgold'] - np.mean(adcs['avg_earnedgold'])) / np.std(adcs['avg_earnedgold'])
adc_earned_gpm_std = (adcs['avg_earnedgpm'] - np.mean(adcs['avg_earnedgpm'])) / np.std(adcs['avg_earnedgpm'])
adc_gold_share_std = (adcs['avg_earnedgoldshare'] - np.mean(adcs['avg_earnedgoldshare'])) / np.std(adcs['avg_earnedgoldshare'])
adc_gold_spent_std = (adcs['avg_goldspent'] - np.mean(adcs['avg_goldspent'])) / np.std(adcs['avg_goldspent'])
adc_cs_std = (adcs['avg_cs'] - np.mean(adcs['avg_cs'])) / np.std(adcs['avg_cs'])
adc_minion_kills_std = (adcs['avg_minionkills'] - np.mean(adcs['avg_minionkills'])) / np.std(adcs['avg_minionkills'])
adc_cspm_std = (adcs['avg_cspm'] - np.mean(adcs['avg_cspm'])) / np.std(adcs['avg_cspm'])
adc_goldat10_std = (adcs['avg_goldat10'] - np.mean(adcs['avg_goldat10'])) / np.std(adcs['avg_goldat10'])
adc_xpat10_std = (adcs['avg_xpat10'] - np.mean(adcs['avg_xpat10'])) / np.std(adcs['avg_xpat10'])
adc_csat10_std = (adcs['avg_csat10'] - np.mean(adcs['avg_csat10'])) / np.std(adcs['avg_csat10'])
adc_golddiffat10_std = (adcs['avg_golddiffat10'] - np.mean(adcs['avg_golddiffat10'])) / np.std(adcs['avg_golddiffat10'])
adc_xpdiffat10_std = (adcs['avg_xpdiffat10'] - np.mean(adcs['avg_xpdiffat10'])) / np.std(adcs['avg_xpdiffat10'])
adc_csdiffat10_std = (adcs['avg_csdiffat10'] - np.mean(adcs['avg_csdiffat10'])) / np.std(adcs['avg_csdiffat10'])
adc_killsat10_std = (adcs['avg_killsat10'] - np.mean(adcs['avg_killsat10'])) / np.std(adcs['avg_killsat10'])
adc_assistsat10_std = (adcs['avg_assistsat10'] - np.mean(adcs['avg_assistsat10'])) / np.std(adcs['avg_assistsat10'])
adc_deathsat10_std = (adcs['avg_deathsat10'] - np.mean(adcs['avg_deathsat10'])) / np.std(adcs['avg_deathsat10'])
adc_goldat15_std = (adcs['avg_goldat15'] - np.mean(adcs['avg_goldat15'])) / np.std(adcs['avg_goldat15'])
adc_xpat15_std = (adcs['avg_xpat15'] - np.mean(adcs['avg_xpat15'])) / np.std(adcs['avg_xpat15'])
adc_csat15_std = (adcs['avg_csat15'] - np.mean(adcs['avg_csat15'])) / np.std(adcs['avg_csat15'])
adc_golddiffat15_std = (adcs['avg_golddiffat15'] - np.mean(adcs['avg_golddiffat15'])) / np.std(adcs['avg_golddiffat15'])
adc_xpdiffat15_std = (adcs['avg_xpdiffat15'] - np.mean(adcs['avg_xpdiffat15'])) / np.std(adcs['avg_xpdiffat15'])
adc_csdiffat15_std = (adcs['avg_csdiffat15'] - np.mean(adcs['avg_csdiffat15'])) / np.std(adcs['avg_csdiffat15'])
adc_killsat15_std = (adcs['avg_killsat15'] - np.mean(adcs['avg_killsat15'])) / np.std(adcs['avg_killsat15'])
adc_assistsat15_std = (adcs['avg_assistsat15'] - np.mean(adcs['avg_assistsat15'])) / np.std(adcs['avg_assistsat15'])
adc_deathsat15_std = (adcs['avg_deathsat15'] - np.mean(adcs['avg_deathsat15'])) / np.std(adcs['avg_deathsat15'])

In [22]:
def top_score(row):
    score = 0
    p = row['player']
    score = score + (2 * top_win_rate_std[p]) + (3 * top_kills_std[p]) - (3 * top_deaths_std[p]) + top_assists_std[p] + (2 * top_first_blood_rate_std[p])
    score = score - (2 * top_first_blood_victim_rate_std[p]) + top_kpm_std[p] + top_dmg_std[p] + (2 * top_dpm_std[p]) + top_dmg_mitigated_std[p]
    score = score + top_wards_placed_std[p] + top_wpm_std[p] + top_wards_killed_std[p] + top_ctrl_wards_std[p] + top_vision_score_std[p]
    score = score + (2 * top_total_gold_std[p]) + (4 * top_earned_gold_std[p]) + (2 * top_earned_gpm_std[p]) + top_gold_share_std[p] + top_gold_spent_std[p]
    score = score + (4 * top_cs_std[p]) + (3 * top_minion_kills_std[p]) + (4 * top_cspm_std[p]) + top_goldat10_std[p] + top_xpat10_std[p] + (3 * top_csat10_std[p])
    score = score + (3 * top_golddiffat10_std[p]) + (3 * top_xpdiffat10_std[p]) + (4 * top_csdiffat10_std[p]) + (3 * top_killsat10_std[p]) + (2 * top_assistsat10_std[p])
    score = score - (3 * top_deathsat10_std[p]) + (3 * top_goldat15_std[p]) + (4 * top_xpat15_std[p]) + (3 * top_csat15_std[p]) + (3 * top_golddiffat15_std[p]) + (4 * top_xpdiffat15_std[p])
    score = score + (4 * top_csdiffat15_std[p]) + (3 * top_killsat15_std[p]) + (2 * top_assistsat15_std[p]) - (3 * top_deathsat15_std[p])
    return score

def mid_score(row):
    score = 0
    p = row['player']
    score = score + (1 * mid_win_rate_std[p]) + (3 * mid_kills_std[p]) - (3 * mid_deaths_std[p]) + mid_assists_std[p] + (2 * mid_first_blood_rate_std[p])
    score = score - (2 * mid_first_blood_victim_rate_std[p]) + mid_kpm_std[p] + mid_dmg_std[p] + (2 * mid_dpm_std[p]) + mid_dmg_mitigated_std[p]
    score = score + mid_wards_placed_std[p] + mid_wpm_std[p] + mid_wards_killed_std[p] + mid_ctrl_wards_std[p] + mid_vision_score_std[p]
    score = score + (2 * mid_total_gold_std[p]) + (4 * mid_earned_gold_std[p]) + (2 * mid_earned_gpm_std[p]) + mid_gold_share_std[p] + mid_gold_spent_std[p]
    score = score + (4 * mid_cs_std[p]) + (3 * mid_minion_kills_std[p]) + (4 * mid_cspm_std[p]) + mid_goldat10_std[p] + mid_xpat10_std[p] + (3 * mid_csat10_std[p])
    score = score + (3 * mid_golddiffat10_std[p]) + (3 * mid_xpdiffat10_std[p]) + (4 * mid_csdiffat10_std[p]) + (4 * mid_killsat10_std[p]) + (2 * mid_assistsat10_std[p])
    score = score - (3 * mid_deathsat10_std[p]) + (3 * mid_goldat15_std[p]) + (4 * mid_xpat15_std[p]) + (3 * mid_csat15_std[p]) + (3 * mid_golddiffat15_std[p]) + (4 * mid_xpdiffat15_std[p])
    score = score + (4 * mid_csdiffat15_std[p]) + (4 * mid_killsat15_std[p]) + (2 * mid_assistsat15_std[p]) - (3 * mid_deathsat15_std[p])
    return score

def jg_score(row):
    score = 0
    p = row['player']
    score = score + (2 * jg_win_rate_std[p]) + (3 * jg_kills_std[p]) - (3 * jg_deaths_std[p]) + (2 * jg_assists_std[p]) + (4 * jg_first_blood_rate_std[p])
    score = score - (4 * jg_first_blood_victim_rate_std[p]) + jg_kpm_std[p] + jg_dmg_std[p] + (2 * jg_dpm_std[p]) + jg_dmg_mitigated_std[p]
    score = score + (2 * jg_wards_placed_std[p]) + (2 * jg_wpm_std[p]) + jg_wards_killed_std[p] + jg_ctrl_wards_std[p] + (3 * jg_vision_score_std[p])
    score = score + (2 * jg_total_gold_std[p]) + (2 * jg_earned_gold_std[p]) + (2 * jg_earned_gpm_std[p]) + jg_gold_share_std[p] + jg_gold_spent_std[p]
    score = score + (3 * jg_cs_std[p]) + (5 * jg_monsterkillsownjung_std[p]) + (5 * jg_monsterkillsenemyjung_std[p]) + jg_goldat10_std[p] + jg_xpat10_std[p] + (1 * jg_csat10_std[p])
    score = score + (3 * jg_golddiffat10_std[p]) + (4 * jg_xpdiffat10_std[p]) + (2 * jg_csdiffat10_std[p]) + jg_killsat10_std[p] + (4 * jg_assistsat10_std[p])
    score = score - (3 * jg_deathsat10_std[p]) + (3 * jg_goldat15_std[p]) + (4 * jg_xpat15_std[p]) + (1 * jg_csat15_std[p]) + (2 * jg_golddiffat15_std[p]) + (4 * jg_xpdiffat15_std[p])
    score = score + (2 * jg_csdiffat15_std[p]) + (3 * jg_killsat15_std[p]) + (2 * jg_assistsat15_std[p]) - (3 * jg_deathsat15_std[p])
    return score

def sup_score(row):
    score = 0
    p = row['player']
    score = score + (2 * sup_win_rate_std[p]) + (1 * sup_kills_std[p]) - (3 * sup_deaths_std[p]) + (3 * sup_assists_std[p]) + (1 * sup_first_blood_rate_std[p])
    score = score - (2 * sup_first_blood_victim_rate_std[p]) + sup_kpm_std[p] + sup_dmg_std[p] + (1 * sup_dpm_std[p]) + (4 * sup_dmg_mitigated_std[p])
    score = score + (3 * sup_wards_placed_std[p]) + (2 * sup_wpm_std[p]) + (3 * sup_wards_killed_std[p]) + (2 * sup_ctrl_wards_std[p]) + (4 * sup_vision_score_std[p])
    score = score + (1 * sup_total_gold_std[p]) + (2 * sup_earned_gold_std[p]) + (2 * sup_earned_gpm_std[p]) + sup_gold_share_std[p] + (2 * sup_gold_spent_std[p])
    score = score + (1 * sup_cs_std[p]) + (1 * sup_minion_kills_std[p]) + (3 * sup_cspm_std[p]) + sup_goldat10_std[p] + sup_xpat10_std[p] + (1 * sup_csat10_std[p])
    score = score + (3 * sup_golddiffat10_std[p]) + (3 * sup_xpdiffat10_std[p]) + (4 * sup_csdiffat10_std[p]) + sup_killsat10_std[p] + (3 * sup_assistsat10_std[p])
    score = score - (3 * sup_deathsat10_std[p]) + (3 * sup_goldat15_std[p]) + (4 * sup_xpat15_std[p]) + (3 * sup_csat15_std[p]) + (3 * sup_golddiffat15_std[p]) + (4 * sup_xpdiffat15_std[p])
    score = score + (4 * sup_csdiffat15_std[p]) + (1 * sup_killsat15_std[p]) + (3 * sup_assistsat15_std[p]) - (3 * sup_deathsat15_std[p])
    return score

def adc_score(row):
    score = 0
    p = row['player']
    score = score + (2 * adc_win_rate_std[p]) + (3 * adc_kills_std[p]) - (3 * adc_deaths_std[p]) + adc_assists_std[p] + (3 * adc_first_blood_rate_std[p])
    score = score - (2 * adc_first_blood_victim_rate_std[p]) + (3 * adc_kpm_std[p]) + (4 * adc_dmg_std[p]) + (4 * adc_dpm_std[p]) + adc_dmg_mitigated_std[p]
    score = score + adc_wards_placed_std[p] + adc_wpm_std[p] + adc_wards_killed_std[p] + adc_ctrl_wards_std[p] + adc_vision_score_std[p]
    score = score + (2 * adc_total_gold_std[p]) + (2 * adc_earned_gold_std[p]) + (2 * adc_earned_gpm_std[p]) + adc_gold_share_std[p] + adc_gold_spent_std[p]
    score = score + (2 * adc_cs_std[p]) + (2 * adc_minion_kills_std[p]) + (2 * adc_cspm_std[p]) + adc_goldat10_std[p] + adc_xpat10_std[p] + (3 * adc_csat10_std[p])
    score = score + (3 * adc_golddiffat10_std[p]) + (3 * adc_xpdiffat10_std[p]) + (4 * adc_csdiffat10_std[p]) + (3 * adc_killsat10_std[p]) + (2 * adc_assistsat10_std[p])
    score = score - (3 * adc_deathsat10_std[p]) + (3 * adc_goldat15_std[p]) + (4 * adc_xpat15_std[p]) + (3 * adc_csat15_std[p]) + (3 * adc_golddiffat15_std[p]) + (4 * adc_xpdiffat15_std[p])
    score = score + (4 * adc_csdiffat15_std[p]) + (4 * adc_killsat15_std[p]) + (2 * adc_assistsat15_std[p]) - (4 * adc_deathsat15_std[p])
    return score


In [23]:
mids['player_score'] = mids.apply(mid_score, axis=1)

In [24]:
tops['player_score'] = tops.apply(top_score, axis=1)

In [25]:
jgs['player_score'] = jgs.apply(jg_score, axis=1)

In [26]:
adcs['player_score'] = adcs.apply(adc_score, axis=1)

In [27]:
sups['player_score'] = sups.apply(sup_score, axis=1)

In [31]:
%store matches
%store mids
%store tops
%store jgs
%store adcs
%store sups
%store lol
%store only_exp

Stored 'matches' (DataFrame)
Stored 'mids' (DataFrame)
Stored 'tops' (DataFrame)
Stored 'jgs' (DataFrame)
Stored 'adcs' (DataFrame)
Stored 'sups' (DataFrame)
Stored 'lol' (DataFrame)
Stored 'only_exp' (DataFrame)
